In [11]:
import requests
import json
import pandas as pd
import numpy as np
import faiss
import ast

In [4]:
df = pd.read_csv(r"new_new_hahaton.csv")

In [5]:
arr = "А если я живу за границей, смогу ли я хоть как-то зарегистрироваться?"

In [15]:
API_URL = "https://llm.t1v.scibox.tech/v1/embeddings"
API_KEY = "sk-h7YcuRfbDw_2fPiuZzz06w"

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

data = {
    "model": "bge-m3",
    "input": arr
}

response = requests.post(API_URL, headers=headers, json=data)
result = response.json()
vect = result["data"][0]["embedding"]


In [12]:
df['emb_question'] = df['emb_question'].apply(ast.literal_eval)

# Теперь можно создать numpy-массив
embeddings_array = np.array(df['emb_question'].tolist(), dtype='float32')

# Размерность эмбеддингов
d = embeddings_array.shape[1]

# Создаём индекс FAISS
index = faiss.IndexFlatL2(d)
index.add(embeddings_array)

# embeddings_array /= np.linalg.norm(embeddings_array, axis=1, keepdims=True)
# vect /= np.linalg.norm(vect)

# # Создаем Faiss индекс для inner product
# d = embeddings_array.shape[1]
# index = faiss.IndexFlatIP(d)
# index.add(embeddings_array)


In [13]:
query_vec = np.array(vect, dtype='float32').reshape(1, -1)
k = 5  # сколько ближайших найти
distances, indices = index.search(query_vec, k)

nearest_categories = df['category'].iloc[indices[0]].tolist()
nearest_subcategories = df['subcategory'].iloc[indices[0]].tolist()
nearest_que = df['question'].iloc[indices[0]].tolist()

# Например, вывод
for cat, subcat, que,dist in zip(nearest_categories, nearest_subcategories, nearest_que,distances[0]):
    print(f"Category: {cat}, Subcategory: {subcat},que: {que}, Distance: {dist}")

Category: Частные клиенты, Subcategory: Кредиты,que: Могу ли я взять кредит, если официально работаю за рубежом?, Distance: 0.5972949862480164
Category: Продукты - Кредиты, Subcategory: Потребительские - Дальше - меньше,que: Можно ли оформить онлайн?, Distance: 0.6015995740890503
Category: Частные клиенты, Subcategory: Онлайн-сервисы,que: Есть ли возможность принимать платежи из-за границы?, Distance: 0.6129500865936279
Category: Продукты - Кредиты, Subcategory: Экспресс-кредиты - В магазинах-партнерах,que: В каких магазинах можно оформить?, Distance: 0.7150251865386963
Category: Продукты - Вклады, Subcategory: Рублевые - СуперСемь,que: Можно ли открыть онлайн?, Distance: 0.7254409790039062


In [ ]:
# query_vec = vect.reshape(1, -1)
# k = 5
# similarities, indices = index.search(query_vec, k)

# # Получаем результаты
# nearest_categories = df['category'].iloc[indices[0]].tolist()
# nearest_subcategories = df['subcategory'].iloc[indices[0]].tolist()
# nearest_questions = df['question'].iloc[indices[0]].tolist()


In [14]:
# for cat, subcat, que, sim in zip(nearest_categories, nearest_subcategories, nearest_questions, similarities[0]):
#     print(f"Category: {cat}, Subcategory: {subcat}, Question: {que}, Cosine similarity: {sim}")